# Baseline Ensemble Methods - Target: Attempted Suicide


## Import Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier


pd.set_option('display.max_columns', 150, 'display.max_rows', 150)

In [2]:
# Read in data and split data to be used in the models
df = pd.read_csv('modeling_dataset.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)

df.head()

,YEAR,raceeth,How old are you,What is your sex,In what grade are you,How tall are you,How much do you weigh,Seat belt use,Riding with a drinking driver,Drinking and driving,Weapon carrying,Weapon carrying at school,Safety concerns at school,Threatened at school,Physical fighting,Physical fighting at school,Forced sexual intercourse,Bullying at school,Electronic bullying,Sad or hopeless,Considered suicide,Made a suicide plan,Attempted suicide,Injurious suicide attempt,Ever cigarette use,Initiation of cigarette smoking,Current cigarette use,Current smokeless tobacco use,Current cigar use,Initiation of alcohol use,Current alcohol use,Source of alcohol,Ever marijuana use,Initiation of marijuana use,Current marijuana use,Ever steroid use,Illegal injected drug use,Illegal drugs at school,Ever sexual intercourse,Sex before 13 years,Multiple sex partners,Current sexual activity,Alcohol/drugs and sex,Condom use,Birth control pill use,Perception of weight,Weight loss,Television watching,Computer use,HIV testing,Asthma,Sleep,Ever used LSD,BMIPCT,weight,stratum,psu,Has used hard drugs,healthy_eating,regular_activity
0,2019.0,7.0,5.0,2.0,2.0,1.63,54.89,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,2.0,4.0,1.0,46.882141,1.6659,213.0,57923.0,0.0,1.0,1.0
1,2019.0,8.0,4.0,2.0,2.0,1.60,53.98,5.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,5.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,4.0,1.0,62.232194,1.3851,213.0,57923.0,0.0,1.0,1.0
2,2019.0,8.0,4.0,1.0,2.0,1.68,43.09,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,2.0,4.0,3.0,2.0,5.0,1.0,0.590171,1.4958,213.0,57923.0,0.0,1.0,0.0
3,2019.0,5.0,4.0,2.0,2.0,1.78,68.95,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,4.0,3.0,1.0,1.0,5.0,3.0,7.0,7.0,4.0,6.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,7.0,7.0,2.0,3.0,4.0,3.0,69.786634,1.7114,213.0,57923.0,1.0,1.0,0.0
4,2019.0,6.0,5.0,2.0,2.0,1.78,58.97,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,1.0,4.0,2.0,2.0,2.0,1.0,16.739994,1.6659,213.0,57923.0,0.0,1.0,0.0


In [3]:
# Create matrix of features
X = df.drop('Attempted suicide', axis = 1) # grabs everything else but target

# Create target variable
y = df['Attempted suicide'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_cols = X.columns

In [4]:
# Use x and y variables to split the training data into train and test set then scale that data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=42)

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## Fit a KNN model

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
knn = KNeighborsClassifier(n_neighbors=9)

In [7]:
knn.fit(X_train, y_train)

knn_preds = knn.predict(X_test)

knn_f1 = metrics.f1_score(y_test, knn_preds, average='weighted')


print(knn_f1)

0.9350091802297837


## Fit a Logistic Regression model 

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
lr = LogisticRegression(class_weight='balanced')

In [10]:
lr.fit(X_train, y_train)

/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [11]:
lr_preds = lr.predict(X_test)

lr_f1 = metrics.f1_score(y_test, lr_preds, average='weighted')

print(lr_f1)

0.9411999749651749


## Fit a Decision Tree Classifier

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
dtc = DecisionTreeClassifier(max_depth=5, class_weight='balanced')

dtc.fit(X_train, y_train)

dtc_preds  = dtc.predict(X_test)

dtc_f1 = metrics.f1_score(y_test, dtc_preds, average='weighted')

print(dtc_f1)

0.930768918387795


## Combine three models using Voting Classifier

In [16]:
from sklearn.ensemble import VotingClassifier


For the estimators, we must provide a list of tuples. The first value in the tuple is is the name given to the model/estimator in the second value. SKlearn requires this because there is additional functionality where you can access information about the specific models, so you need to name the models to access them later.  

In [19]:
voting_clf = VotingClassifier(
                estimators=[('logreg', lr), ('knneighbors', knn), ('decisiontree', dtc)], 
                voting='hard')

voting_clf.fit(X_train, y_train)

vc_preds = voting_clf.predict(X_test)

vc_f1 = metrics.f1_score(y_test, vc_preds, average='weighted')

print(vc_f1)

/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9421721640119471


### Use a voting classifier with multiple Logistic regression models 

In [21]:

C_param_range = [0.001,0.01,0.1,1,10]
titles = ['lr_0_001', 'lr_0_01', 'lr_0_1', 'lr_1', 'lr_10']

params = dict(zip(titles, C_param_range)) 
models = {}

table = pd.DataFrame(columns = ['C_parameter','F1'])
table['C_parameter'] = C_param_range
j = 0

for k , v  in params.items():
    
    # Create model using different value for c  
    lr = LogisticRegression(penalty = 'l2', C = v, random_state = 1, class_weight='balanced')
    
    #save the model to a dictionary to use later in our voting classifiers
    models[k]= lr
    
    #the steps below this point are unnecessary in order to create a voting classifier, 
    #but it is easy to fit the model and see how performance changes for different levels of regularization
    lr.fit(X_train, y_train)
    
    # Predict using model
    y_preds = lr.predict(X_test)

    # Saving accuracy score in table
    table.iloc[j,1] = metrics.f1_score(y_test, y_preds, average='weighted')
    j += 1



/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

In [22]:
#review performance for different levels of C
table


,C_parameter,F1
0,0.001,0.941635
1,0.010,0.942353
2,0.100,0.941711
3,1.000,0.9412
4,10.000,0.941564


In [23]:
#invesitgate the models D=dictionary
list(models.items())

[('lr_0_001',
  LogisticRegression(C=0.001, class_weight='balanced', random_state=1)),
 ('lr_0_01',
  LogisticRegression(C=0.01, class_weight='balanced', random_state=1)),
 ('lr_0_1',
  LogisticRegression(C=0.1, class_weight='balanced', random_state=1)),
 ('lr_1', LogisticRegression(C=1, class_weight='balanced', random_state=1)),
 ('lr_10', LogisticRegression(C=10, class_weight='balanced', random_state=1))]

Now that we have programmatically created multiple logistic regression models, let's use them in an ensemble model

In [24]:
lr_voting = VotingClassifier(estimators=list(models.items()), 
                              voting='hard')

lr_voting.fit(X_train, y_train)

lrv_preds = lr_voting.predict(X_test)

lrv_f1 = metrics.f1_score(y_test, lrv_preds, average='weighted')

print(lrv_f1)

/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

0.9418343691647996


/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Fit a Bagging Classifier for a Logistic Regression model. 

In [25]:
X_train.shape

(40161, 59)

In [26]:
bc_lr = BaggingClassifier(
            base_estimator=LogisticRegression(random_state = 1, class_weight='balanced'), 
            n_estimators= 100,
            max_samples= 0.8,
            max_features= 6,
            oob_score= True
                )

In [27]:
bc_lr.fit(X_train, y_train)



/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

BaggingClassifier(base_estimator=LogisticRegression(class_weight='balanced',
                                                    random_state=1),
                  max_features=6, max_samples=0.8, n_estimators=100,
                  oob_score=True)

In [28]:
# Use the oob_score to get some idea of how the model performs on a validation set

bc_lr.oob_score_

0.922711087871318

In [31]:
# See how the model performs on the test set

bc_lr_preds = bc_lr.predict(X_test)

bc_lr_f1 = metrics.f1_score(y_test, bc_lr_preds, average='weighted')

print(bc_lr_f1)

0.9273726459956455


***What is the difference in the `VotingClassifier` algorithm and the `BaggingClassifier` algorithm?***

Your answer:

**What is the difference between a BaggingClassifier that uses a decision tree as the base estimator and a Random Forest Classifier?**

A random forest classifier will take a sample of features at each node, where as a bagging classifier will take a sample of features at to use for the whole model. 

# Fitting a Random Forest Classifier

In [32]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=1000)

In [33]:
#let's look at all the different default features
rfc

RandomForestClassifier(n_estimators=1000, random_state=1)

In [34]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=1)

In [35]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)

rfc_f1 = metrics.f1_score(y_test, rfc_preds, average='weighted')

# checking accuracy on the test data
print('Test F1 score: ', rfc_f1)

Test F1 score:  0.9518437458926547


***Change the parameters and see whether you can improve the performance of the model***

### GridsearchCV with Random Forest

Let's use grid search to identify the best tuning parameters to use for a random forest model. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
RandomForestClassifier()

In [ ]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    'n_estimators': [100,300,500,700,1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(2,10)),
    'max_features': list(range(3,7))
}

In [ ]:
#create a grid search object and fit it to the data

grid_tree=GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

In [ ]:
grid_tree.fit(X_train, y_train)

In [ ]:
### Identify the best params 



# Single best score achieved across all params (min_samples_split)
print(grid_tree.best_score_)

# Dictionary containing the parameters (min_samples_split) used to generate that score
print(grid_tree.best_params_)

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print(grid_tree.best_estimator_)
#Identify the best score during fitting with cross-validation


In [ ]:
#Predict the response for test dataset
y_pred = grid_tree.best_estimator_.predict(X_test)

# Model F1, how often is the classifier correct?
print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))

# Grid Search and Pipelines

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
#Load the dataset
df = sns.load_dataset('titanic')

In [3]:
#Drop nulls in embarked
df.dropna(subset=['embarked','embark_town'], inplace=True)

In [4]:
cols = ['pclass', 'sex','parch', 
       'embarked', 'class', 'who', 'adult_male', 'embark_town']

In [5]:
#Fit the 
le = LabelEncoder()
df.loc[:,cols] = df.loc[:,cols].apply(le.fit_transform)

In [6]:
# define x and y 
y = df['survived']
X = df[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town']]

In [7]:
X.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town
0,2,1,22.0,1,0,7.2500,2,2,1,1,2
1,0,0,38.0,1,0,71.2833,0,0,2,0,0
2,2,0,26.0,0,0,7.9250,2,2,2,0,2
3,0,0,35.0,1,0,53.1000,2,0,2,0,2
4,2,1,35.0,0,0,8.0500,2,2,1,1,2


We can split the data using stratify to make sure we can get an even split of classes in the train and test set

In [8]:
# Split X and y with even class distributions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [9]:
#Impute age using the training mean
mean_age = X_train.age.mean()
X_train['age'].fillna(mean_age, inplace=True)
X_test['age'].fillna(mean_age, inplace=True)

/Users/sandrawelbeck/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# Random Forest

In [10]:
#Scale the data
ss = StandardScaler()
ss.fit(X_train)
X_train = pd.DataFrame(ss.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

In [11]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7808988764044944

In [12]:
#Now with cross validation search
val = cross_val_score(RandomForestClassifier(random_state=42),X_train,y_train,cv=5)
val.mean()

0.8002954791687186

# Random Forest with Grid Search

In [13]:
#If you were to line up all the possible values of one and of the other, they create a grid

In [14]:
rf_param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16],
    "max_features": [None,4,5,6,9,10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf" : [1, 2, 3, 5, 6],
    "n_estimators" : [10, 30, 100]
}

In [15]:
rf_grid = RandomForestClassifier(random_state=42)
gridsearch = GridSearchCV(rf_grid, rf_param_grid, cv=5, return_train_score=True, n_jobs=-1, verbose=-1)

In [16]:
gridsearch.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 1144 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 3520 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 4600 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 8704 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 10360 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done 12160 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 14104 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done 16192 tasks   

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16],
                         'max_features': [None, 4, 5, 6, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 5, 6],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 30, 100]},
             return_train_score=True, verbose=-1)

In [17]:
print("Testing Accuracy: {:.4}%".format(gridsearch.best_score_ * 100))
print("")
print("Optimal Parameters: {}".format(gridsearch.best_params_))

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print("Best Model: {}".format(gridsearch.best_estimator_))

Testing Accuracy: 84.11%

Optimal Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 5, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 10}
Best Model: RandomForestClassifier(max_depth=10, max_features=5, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=10, random_state=42)


In [18]:
gridsearch.score(X_test, y_test)

0.8089887640449438

In [19]:
#CV * Number of params to search
5*2*11*6*3*5*3

29700

# Pipelines

In [174]:
# Split X and y with even class distributions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

#Impute age using the training mean
mean_age = X_train.age.mean()
X_train['age'].fillna(mean_age, inplace=True)
X_test['age'].fillna(mean_age, inplace=True)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [175]:
# Create our pipeline by passing a list of tuples with each class and its name in a string
pipeline1 = Pipeline([
                    ('ss',StandardScaler()),
                    ('rf', RandomForestClassifier(random_state=42))
                    ])

In [176]:
# Fit the training data to pipeline
pipeline1.fit(X_train,y_train)

# Print the accuracy on test set
pipeline1.score(X_test,y_test)

0.7808988764044944

# Pipelines with GridSearch

In [177]:
pipeline2 = Pipeline([
                    ('ss',StandardScaler()),
                    ('rf', RandomForestClassifier(random_state=42))
                    ])

In [178]:
# We set the params, but we add a rf__ to each parameter to let it know its on the rf part of the pipeline
rf_param_grid_pipe = {
    "rf__criterion": ["gini", "entropy"],
    "rf__max_depth": [None, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16],
    "rf__max_features": [None,4,5,6,9,10],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf" : [1, 2, 3, 5, 6],
    "rf__n_estimators" : [10, 30, 100]
}

In [179]:
# Instanstiate the gridsearch pipeline
grid_search_pipe = GridSearchCV(pipeline2, rf_param_grid_pipe, cv=5, verbose=1, n_jobs=-1, return_train_score=True)


In [180]:
#Fit the grid search to the data
grid_search_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 5940 candidates, totalling 29700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 352 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 852 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 2452 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 3552 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 4852 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 6352 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 8052 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 9952 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 12052 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 14352 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 16852 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 19552 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 22452 tasks  

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('rf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_depth': [None, 2, 3, 4, 5, 6, 8, 10, 12, 14,
                                           16],
                         'rf__max_features': [None, 4, 5, 6, 9, 10],
                         'rf__min_samples_leaf': [1, 2, 3, 5, 6],
                         'rf__min_samples_split': [2, 5, 10],
                         'rf__n_estimators': [10, 30, 100]},
             return_train_score=True, verbose=1)

In [181]:
grid_search_pipe.score(X_test, y_test)

0.8089887640449438

In [182]:

# Single best score achieved across all params (min_samples_split)
print(grid_search_pipe.best_score_)

# Dictionary containing the parameters (min_samples_split) used to generate that score
print(grid_search_pipe.best_params_)

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print(grid_search_pipe.best_estimator_)
#Identify the best score during fitting with cross-validation

0.8411110016743819
{'rf__criterion': 'gini', 'rf__max_depth': 10, 'rf__max_features': 5, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 10, 'rf__n_estimators': 10}
Pipeline(steps=[('ss', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=10, max_features=5,
                                        min_samples_leaf=3,
                                        min_samples_split=10, n_estimators=10,
                                        random_state=42))])


In [183]:
#Predictions with grid search
#Predict the response for test dataset
y_pred = grid_search_pipe.best_estimator_.predict(X_test)

# Model F1, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.8089887640449438


### Pickling the best Estimator



In [184]:
# pickle list object
 
model_pickle_path = 'best_model.pkl'

#Create a variable to pickle and open it in write mode
model_pickle = open(model_pickle_path, 'wb')
pickle.dump(grid_search_pipe.best_estimator_, model_pickle)
model_pickle.close()

# Randomized Search

Randomized Search allows us to search a distribution for each parameter instead of only a desginated value like GridSearch



In [185]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

#Load in the Iris Dataset and the LogisticRegression class
iris = load_iris()
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)
# Set a distribution and options for the different parameters
distributions = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])
#Fit and check the best parameters
clf = RandomizedSearchCV(logistic, distributions, random_state=0)
search = clf.fit(iris.data, iris.target)
search.best_params_

{'C': 2.195254015709299, 'penalty': 'l1'}

In [186]:
search.score(iris.data, iris.target)

0.98